In [1]:
import numpy as np
import open3d as o3d
import cv2
import PIL
import os
import copy
import time

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import pygicp

In [3]:
def draw_registration_result(source, target, transformation=np.eye(4)):
    source_temp= copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],)

In [4]:
focalX = 1040.29
focalY = 1040.5
centerX = 632
centerY = 416
scalingFactor = 1.0

def readDepth(path):
	min_depth_percentile = 5
	max_depth_percentile = 95

	with open(path, "rb") as fid:
		width, height, channels = np.genfromtxt(fid, delimiter="&", max_rows=1, usecols=(0, 1, 2), dtype=int)
		print(width,height)
		fid.seek(0)
		num_delimiter = 0
		byte = fid.read(1)
		while True:
			if byte == b"&":
				num_delimiter += 1
				if num_delimiter >= 3:
					break
			byte = fid.read(1)
		array = np.fromfile(fid, np.float32)
	array = array.reshape((width, height, channels), order="F")

	depth_map = np.transpose(array, (1, 0, 2)).squeeze()

	min_depth, max_depth = np.percentile(depth_map, [min_depth_percentile, max_depth_percentile])
	print(min_depth, max_depth)

# 	depth_map[depth_map < min_depth] = min_depth
# 	depth_map[depth_map > max_depth] = max_depth

	return depth_map

def getPointCloud(rgbFile, depthFile):
	thresh = 15.0

	depth = readDepth(depthFile)
	rgb = PIL.Image.open(rgbFile)
	points = []
	colors = []
	srcPxs = []
	print(depth.shape)
	for v in range(depth.shape[0]):
		for u in range(depth.shape[1]):
			
			Z = depth[v, u] / scalingFactor
			if Z==0: continue
			if (Z > thresh): continue

			X = (u - centerX) * Z / focalX
			Y = (v - centerY) * Z / focalY
			
			srcPxs.append((u, v))
			points.append((X, Y, Z))
			colors.append(rgb.getpixel((u, v)))

	srcPxs = np.asarray(srcPxs).T
	points = np.asarray(points)
	colors = np.asarray(colors)
	
	pcd = o3d.geometry.PointCloud()
	pcd.points = o3d.utility.Vector3dVector(points)
	pcd.colors = o3d.utility.Vector3dVector(colors/255)
	
	return pcd, srcPxs, depth

In [5]:
def getPointCloudFast(rgbFile, depthFile, sampling_ratio=0.1):
    depth = readDepth(depthFile)
    rgb = PIL.Image.open(rgbFile)
    width, height = depth.shape[1], depth.shape[0]
    u_list = np.tile([i for i in range(width)], height).reshape(height, width)
    v_list = np.tile([j for j in range(height)], width).reshape(width, height).transpose()
    X = (u_list - centerX) * depth / focalX
    Y = (v_list - centerY) * depth / focalY
    points = np.stack([X,Y,depth], axis=-1)
    points = np.reshape(points, [-1, 3])
    colors = np.array(rgb).reshape([-1,3])
    points_num = len(points)
    print("points_num", points_num)
    selected_indices = np.random.choice(points_num, size=(int)(points_num*sampling_ratio), replace=False)
    points = points[selected_indices]
    colors = colors[selected_indices]
    print("sampled_num", len(points))    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors/255)
    return pcd, depth

In [6]:
pcd = o3d.io.read_point_cloud('251370668.pcd')
pcd1 = o3d.io.read_point_cloud('251371071.pcd')

In [7]:
pcd, depth = getPointCloudFast('DSC05573.jpg', 'DSC05573.bin')
pcd1, depth1 = getPointCloudFast('DSC05572.jpg', 'DSC05572.bin')

1264 832
2.5367496490478514 8.600763320922852
points_num 1051648
sampled_num 105164
1264 832
3.2988741874694822 8.909795761108398
points_num 1051648
sampled_num 105164


In [16]:
depth_img = depth / np.max(depth)*255.
cv2.imwrite('depth.png', depth_img)

True

In [127]:
a = o3d.geometry.PointCloud()
a.points = o3d.utility.Vector3dVector(np.asarray(pcd.points))
b = o3d.geometry.PointCloud()
b.points = o3d.utility.Vector3dVector(np.asarray(pcd1.points))
draw_registration_result(a,b)

In [8]:
gicp = pygicp.FastGICP()

# pcd_points = pygicp.downsample(np.asarray(pcd.points), 0.75)
# pcd1_points = pygicp.downsample(np.asarray(pcd1.points), 0.75)
pcd_points = np.asarray(pcd.points)
pcd1_points = np.asarray(pcd1.points)[0:-10]

gicp.set_input_target(pcd1_points)
gicp.set_input_source(pcd_points)

# gicp.set_resolution(0.3)
# gicp.set_neighbor_search_method("DIRECT7")
# gicp.set_num_threads(1)


set input target end
set input source end


In [9]:
gicp.calculate_source_covariance()
gicp.calculate_target_covariance()

In [10]:
source_scales = gicp.get_source_scales()
target_scales = gicp.get_target_scales()
scales = np.concatenate([source_scales, target_scales], axis=0)
scales = np.reshape(scales, (-1,3)) + 1e-7
scales = scales.astype(np.float32)

target and quaternions size mismatch. Did you change target?


In [138]:
indices = np.where(scales[:,0]/scales[:,1]<2.3, True, False)
indices = indices * np.where(scales[:,1]>1e-3, True, False)
selected_scales = scales[indices]

In [139]:
print(selected_scales.shape)
print(indices.shape)

(206771, 3)
(210328,)


In [124]:
log_scales = np.log(scales)
scales = np.exp(log_scales)

In [141]:
print(np.median(scales[:,0]))
print(np.median(scales[:,1]))
print(np.median(scales[:,2]))
print(np.mean(scales[:,0]))
print(np.mean(scales[:,1]))
print(np.mean(scales[:,2]))
print(np.min(scales[:,0]))
print(np.min(scales[:,1]))
print(np.min(scales[:,2]))
print(np.max(scales[:,0]))
args = np.argsort( -np.abs((scales[:,0]*scales[:,0])/(scales[:,1]*scales[:,1])))

0.038745627
0.029948622
0.0038737222
0.03912044
0.029681357
0.005187797
1e-07
1e-07
1e-07
0.42581514


In [126]:
for arg in args[:1000]:
    print(scales[arg,0]/scales[arg,1])

11.45584
10.923559
10.923559
10.858066
10.262848
9.977767
9.871103
9.228992
9.228992
8.866155
8.517236
8.363424
8.363424
8.204233
8.204233
8.141833
7.5234756
7.3224316
7.033073
6.9338984
6.902673
6.902673
6.7046065
6.7046065
6.4506364
6.3961773
6.3961773
6.299106
6.153973
6.153973
6.051037
6.0097594
5.9909387
5.904088
5.7120914
5.5703864
5.5444484
5.536297
5.5225787
5.5181484
5.4835496
5.478815
5.459139
5.4270763
5.3566008
5.3566008
5.3191285
5.301642
5.279076
5.279076
5.245393
5.1720448
5.1523094
5.1100745
5.1100745
5.1100745
5.102842
5.08843
5.0854154
5.0821996
5.0763793
5.073324
5.052387
4.985967
4.925241
4.8945756
4.865846
4.865846
4.865846
4.865846
4.824024
4.8229628
4.7853065
4.7853065
4.7621193
4.7404103
4.725811
4.71154
4.7084746
4.680385
4.680385
4.680385
4.6759152
4.6735315
4.638246
4.6204467
4.6204467
4.6131425
4.602487
4.5733023
4.5438366
4.543296
4.543296
4.543296
4.538027
4.538027
4.4820657
4.4820657
4.4820657
4.4820657
4.4820657
4.4820657
4.4820657
4.4820657
4.4820657
4.

In [47]:
class a():
    @property
    def get(self):
        return 0
    def ab(self):
        return self.get

In [48]:
a1 = a()

In [49]:
a1.ab()

0

In [16]:
tp = time.time()
for i in range(1000):
    scales = gicp.get_source_scales()
print(time.time()-tp)
tp = time.time()
for i in range(1000):
    rotationsq = gicp.get_source_rotationsq()
print(time.time()-tp)

0.022774696350097656
0.027390718460083008


In [15]:
tp = time.time()
for i in range(1000):
    gicp.set_source_covariances_fromqs(rotationsq, scales)
print(time.time()-tp)

21.79923367500305


In [16]:
a = np.array([i for i in range(100*4)])

In [17]:
a = np.reshape(a, (100,4))

In [19]:
a = a.flatten()

In [ ]:
gicp.se

In [10]:
scales = np.ones((100,3)).tolist()
qs = np.ones((100,4)).tolist()
gicp.set_source_covariances_fromqs(qs, scales)
q = gicp.get_source_rotationsq()
s = gicp.get_source_scales()
print(q[2], s[2])

[1. 1. 1. 1.] [1. 1. 1.]


In [10]:
gicp.calculate_source_covariance()

In [11]:
scales = gicp.get_source_scales()

In [12]:
print(len(scales))

105164


In [13]:
np.min(scales)

0.03162277660168379

In [14]:
for i in range(1000):
    print(scales[i])

[1.46476377 1.         0.20994904]
[1.70220426 1.         0.03162278]
[1.39428003 1.         0.10922772]
[1.223799   1.         0.10899091]
[1.14727335 1.         0.12534965]
[1.40164171 1.         0.03162278]
[1.60261689 1.         0.34538282]
[1.03741207 1.         0.06996643]
[1.1614269  1.         0.24771299]
[1.24200638 1.         0.2258153 ]
[1.22522748 1.         0.05244714]
[1.30923542 1.         0.19438873]
[1.03976095 1.         0.14872119]
[1.08319155 1.         0.14412151]
[1.7793894 1.        0.4992205]
[1.13080694 1.         0.03162278]
[1.43389866 1.         0.09261711]
[1.62906742 1.         0.05566197]
[1.12354181 1.         0.09159286]
[1.20503126 1.         0.34748496]
[1.03800045 1.         0.03162278]
[1.29554453 1.         0.14436235]
[1.11988532 1.         0.1301937 ]
[1.39276608 1.         0.33285593]
[1.5912339  1.         0.24132355]
[1.28444428 1.         0.03162278]
[1.36849501 1.         0.53615388]
[1.61220181 1.         0.20546093]
[1.39687501 1.         

In [10]:
transform = gicp.align()

In [12]:
a = time.time()
for i in range(100):
    correspondences, sq_distances = gicp.get_source_correspondence()
print(time.time()-a)
print(len(correspondences))

0.0032002925872802734
105164


In [13]:
print(correspondences.shape)
print(sq_distances.shape)
print(pcd1_points.shape)
print(pcd_points.shape)

(105164,)
(105164,)
(105154, 3)
(105164, 3)


In [20]:
print(np.max(sq_distances))

2.376938819885254


In [17]:
for i in range(100):
    print(correspondences[i])

76134
71052
39390
77272
74891
10127
26699
60182
66260
17099
4059
2049
1605
96728
2199
10876
72039
48266
100840
9148
92086
53203
52628
7067
14143
53449
81416
29283
64439
25398
88288
90632
3393
41615
59440
9506
99113
85654
12270
17513
14533
4331
57576
98017
71463
103398
73425
39213
25411
80588
59625
82756
97986
66440
19728
3244
41878
12189
94228
91506
54084
22673
86252
55967
19805
39390
982
15781
5873
71660
90609
42096
95951
39390
8980
64687
7708
19296
71810
61373
7226
54162
74212
79496
50502
34198
53321
102922
53125
55679
77962
103277
19278
20891
88196
20743
35098
49915
64942
21162


In [1]:
r = gicp.get_source_rotationsq()

In [10]:
s = gicp.get_source_scales()

In [11]:
s

[array([0.04150902, 0.01617555, 0.00227955]),
 array([0.03316687, 0.02460116, 0.00338104]),
 array([0.04446644, 0.00209123, 0.00015679]),
 array([0.03352654, 0.02733078, 0.00402952]),
 array([0.05043857, 0.01421892, 0.00365149]),
 array([0.0332258 , 0.02748676, 0.00277276]),
 array([0.04590693, 0.02713396, 0.00266119]),
 array([0.03521828, 0.02948238, 0.00403688]),
 array([0.03799838, 0.0203053 , 0.00182111]),
 array([0.03517718, 0.02119787, 0.00333427]),
 array([0.03838311, 0.02115166, 0.00266959]),
 array([0.03188249, 0.02640451, 0.00437315]),
 array([0.03833295, 0.02145919, 0.00248558]),
 array([0.03239741, 0.02364535, 0.00367941]),
 array([0.03562459, 0.02424306, 0.00227075]),
 array([0.03459811, 0.0251032 , 0.00376617]),
 array([0.04143717, 0.01482674, 0.00242674]),
 array([0.03155483, 0.03116   , 0.0031926 ]),
 array([0.03498382, 0.02263852, 0.00188343]),
 array([0.03466346, 0.02754864, 0.0033308 ]),
 array([0.03542611, 0.0242183 , 0.00270299]),
 array([0.03448309, 0.02820308, 0.

In [9]:
source_covariance = gicp.get_source_covariances()

In [10]:
print(source_covariance[0:100])

[array([[ 0.93758362,  0.23885067, -0.03753018,  0.        ],
       [ 0.23885067,  0.0859828 ,  0.14361789,  0.        ],
       [-0.03753018,  0.14361789,  0.97743358,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]]), array([[ 0.91816034,  0.27279015, -0.02540914,  0.        ],
       [ 0.27279015,  0.09072855,  0.08469443,  0.        ],
       [-0.02540914,  0.08469443,  0.99211111,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]]), array([[ 0.9999929 , -0.00128944, -0.00233107,  0.        ],
       [-0.00128944,  0.76594596, -0.42312768,  0.        ],
       [-0.00233107, -0.42312768,  0.23506115,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]]), array([[ 0.90141734,  0.29452641, -0.04494148,  0.        ],
       [ 0.29452641,  0.12007041,  0.13426754,  0.        ],
       [-0.04494148,  0.13426754,  0.97951225,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]]), array([[ 0.9199

In [27]:
test_covlist = []
for i in range(2):
    test_covlist.append(np.ones((3,3)))

In [28]:
gicp.set_source_covariances_from3x3(test_covlist)

In [29]:
source_covariance = gicp.get_source_covariances()

In [33]:
print(len(source_covariance))
print(source_covariance[0:100])

2
[array([[1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [0., 0., 0., 0.]]), array([[1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [0., 0., 0., 0.]])]


In [73]:
transform = pygicp.align_points(target=pcd1_points, source=pcd_points, method='VGICP')

In [11]:
transform = pygicp.align_points(target=pcd1_points, source=pcd_points, method='GICP')

set input target end
set input source end
compute source cov
compute target cov


In [12]:
R = transform[:3,:3]
T = transform[:3,3]

In [15]:
c = o3d.geometry.PointCloud()
c.points = o3d.utility.Vector3dVector(np.matmul(R, pcd_points.transpose()).transpose() + T)
draw_registration_result(c,b)

In [11]:
q,w = gicp.get_voxel_mean_cov()
q = np.array(q)
w = np.array(w)

AttributeError: 'pygicp.FastGICP' object has no attribute 'get_voxel_mean_cov'

In [87]:
d = o3d.geometry.PointCloud()
d.points = o3d.utility.Vector3dVector(np.asarray(q[...,0:3]))
draw_registration_result(c,d)

In [ ]:
    def create_from_pcd(self, pcd : BasicPointCloud, spatial_lr_scale : float):
        self.spatial_lr_scale = spatial_lr_scale
        fused_point_cloud = torch.tensor(np.asarray(pcd.points)).float().cuda()
        fused_color = RGB2SH(torch.tensor(np.asarray(pcd.colors)).float().cuda())
        features = torch.zeros((fused_color.shape[0], 3, (self.max_sh_degree + 1) ** 2)).float().cuda()
        features[:, :3, 0 ] = fused_color
        features[:, 3:, 1:] = 0.0

        print("Number of points at initialisation : ", fused_point_cloud.shape[0])

        gicp.set_input_source(np.asarray(pcd.points))
        scales = torch.from_numpy(gicp.get_source_scales()).cuda()
        # gaussian splatting uses log scale rots; exp will be applied as an activation func when returning this value
        rots = torch.log(torch.from_numpy(gicp.get_source_rotationsq())).cuda()

    #     dist2 = torch.clamp_min(distCUDA2(torch.from_numpy(np.asarray(pcd.points)).float().cuda()), 0.0000001)
    #     scales = torch.log(torch.sqrt(dist2))[...,None].repeat(1, 3)
    #     rots = torch.zeros((fused_point_cloud.shape[0], 4), device="cuda")
    #     # I will follow x = q.x, y=q.y, z=q.z, 1=q.w
    #     # I also modify computeCov3D in RGBD_gaussian-splatting/submodules/diff-gaussian-rasterization/cuda_rasterizer/forward.cu
    #     rots[:, -1] = 1 #  was rots[:, 0] = 1 in original gaussian splatting, dont know why

        opacities = inverse_sigmoid(0.1 * torch.ones((fused_point_cloud.shape[0], 1), dtype=torch.float, device="cuda"))

        self._xyz = nn.Parameter(fused_point_cloud.requires_grad_(True))
        self._features_dc = nn.Parameter(features[:,:,0:1].transpose(1, 2).contiguous().requires_grad_(True))
        self._features_rest = nn.Parameter(features[:,:,1:].transpose(1, 2).contiguous().requires_grad_(True))
        self._scaling = nn.Parameter(scales.requires_grad_(True))
        self._rotation = nn.Parameter(rots.requires_grad_(True))
        self._opacity = nn.Parameter(opacities.requires_grad_(True))
        self.max_radii2D = torch.zeros((self.get_xyz.shape[0]), device="cuda")

In [19]:
import torch
a = torch.nn.Parameter(torch.from_numpy(np.zeros((100,1))).cuda().requires_grad_(True))
b = torch.from_numpy(np.zeros((100,1))).cuda()
c = torch.cat([a,b], axis=-1)


In [20]:
torch.log(np.array([1,2]))

TypeError: log(): argument 'input' (position 1) must be Tensor, not numpy.ndarray